In [1]:
import dash
from dash import Dash, dcc, html, dcc, callback, Output, Input
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import re
import dash_cytoscape as cyto
from functools import reduce
app = Dash(__name__)

In [2]:
# LinkedIn Digital Data
df_ldd = pd.read_excel('..\Data\LinkedIn_DigitalData\public_use-industry-skills-needs.xlsx', sheet_name=['Industry Skills Needs'])['Industry Skills Needs']
df_ldd

,year,isic_section_index,isic_section_name,industry_name,skill_group_category,skill_group_name,skill_group_rank
0,2015,B,Mining and quarrying,Mining & Metals,Specialized Industry Skills,Mining,1
1,2015,B,Mining and quarrying,Mining & Metals,Soft Skills,Negotiation,2
2,2015,B,Mining and quarrying,Mining & Metals,Business Skills,Project Management,3
3,2015,B,Mining and quarrying,Mining & Metals,Business Skills,Business Management,4
4,2015,B,Mining and quarrying,Mining & Metals,Specialized Industry Skills,Earth Science,5
...,...,...,...,...,...,...,...
3495,2019,R,"Arts, entertainment and recreation",Animation,Tech Skills,Social Media,6
3496,2019,R,"Arts, entertainment and recreation",Animation,Tech Skills,Digital Literacy,7
3497,2019,R,"Arts, entertainment and recreation",Animation,Soft Skills,Teamwork,8
3498,2019,R,"Arts, entertainment and recreation",Animation,Specialized Industry Skills,Editing,9


In [3]:
# styles
colors = {
    'background': '#d9ffb3',
    'text': '#0066cc',
    'title' : '#000066'
}
def_stylesheet = [
        {
            'selector': 'node',
            'style': {
                'background-color': 'grey',
                'opacity': '0.2',
                'label': 'data(label)'# Default color for nodes
            }
        },
        {
            'selector': 'node[type = \'industry\']',
            'style': {
                'background-color': '#ff66cc',
                'opacity': '0.2',
                'label': 'data(label)'# Default color for nodes
            }
        },
        {
            'selector': 'edge',
            'style': {
                'line-color': 'grey',  # Default color for edges
                'opacity':'0.15'
            }
        }]

In [4]:
@app.callback(
    [Output("cytoscape", "elements"),
     Output('cytoscape', 'stylesheet')],
    [Input("sector-filter-1", "value"),
     Input("skill-filter-1", "value"),
     Input('cytoscape', 'tapNodeData')]
)
def update_style(section, skill, selected_node):
    filtered_df = df_ldd[df_ldd['isic_section_name']==section]
    if skill:
        filtered_df = filtered_df[filtered_df["skill_group_name"].isin(skill)]
    industries = [
        {
            "data":{"id": iname+"i", "label": iname, "type":'industry'}
        }
        for iname in filtered_df['industry_name'].unique()
    ]
    skills = [
        {
            "data":{"id": sname+"s", "label": sname, "type":'skill'}
        }
        for sname in filtered_df['skill_group_name'].unique()
    ]
    nodes = industries+skills
    edges = [
        {"data": {"source": source+"i", "target":target+"s"}}
        for source, target in list(zip(filtered_df['industry_name'], filtered_df['skill_group_name']))
    ]
    elements = nodes + edges
    if selected_node is None:
        return elements, []

    selected_node_id = selected_node['id']
    neighborhood = []
    for edge in edges:
        edge_data = edge.get('data', {})
        source = edge_data.get('source')
        target = edge_data.get('target')
        if source == selected_node_id:
            neighborhood.append(target)
        elif target == selected_node_id:
            neighborhood.append(source)

    # Define the stylesheet to highlight the selected node and its immediate neighbors
    stylesheet = [
        {
            'selector': 'node',
            'style': {
                'background-color': 'grey',
                'opacity': '0.2',
                'label': 'data(label)'# Default color for nodes
            }
        },
        {
            'selector': 'node[type = \'industry\']',
            'style': {
                'background-color': '#ff66cc',
                'opacity': '0.2',
                'label': 'data(label)'# Default color for nodes
            }
        },
        {
            'selector': 'edge',
            'style': {
                'line-color': 'grey',  # Default color for edges
                'opacity':'0.15'
            }
        },
        {
            'selector': 'node[id = "{}"]'.format(selected_node_id),
            'style': {
                'background-color': 'blue',  # Color for selected node
                'opacity': '1.0'
            }
        },
        {
            'selector': 'edge[source = "{}"]'.format(selected_node_id),
            'style': {
                'line-color': 'blue',  # Color for edges connected to selected node
                'opacity': '1.0'
            }
        },
        {
            'selector': 'edge[target = "{}"]'.format(selected_node_id),
            'style': {
                'line-color': 'blue',  # Color for edges connected to selected node
                'opacity': '1.0'
            }
        }
    ]
    for nd in neighborhood:
        stylesheet.append({
            'selector': 'node[id = "{}"]'.format(nd),
            'style': {
                'background-color': 'red',  # Color for edges connected to selected node
                'opacity': '1.0'
            }})
    return elements,stylesheet

In [5]:
app.layout = html.Div(style={'backgroundColor': colors['background'], 'margin': 10, 'padding': 20}, children=[
    html.H1(
        children='Industry Skill Needs',
        style={
            'textAlign': 'center',
            'color': colors['title']
        }
    ),

     dbc.Row([
         html.Div([
                html.Label("Sector", style={'color': colors['text']}),
                dcc.Dropdown(
                    id="sector-filter-1",
                    options=[{"label": s, "value": s}
                             for s in df_ldd['isic_section_name'].dropna().unique()],
                    value='Information and communication',
                    style={
                        "width": "75%",  # set the width of the dropdown to 50% of its container
                        "height": "50%",  # set the height of each dropdown item to 40 pixels
                        "backgroundColor": colors['background'],  # set the background color of the dropdown to dark gray
                        "color": "#000",  # set the font color of the dropdown to light gray
                        "fontFamily": "sans-serif"  # set the font family of the dropdown to sans-serif
                    }
                ),
             html.Label("Skills", style={'color': colors['text']}),
                dcc.Dropdown(
                    id="skill-filter-1",
                    options=[{"label": s, "value": s}
                             for s in df_ldd['skill_group_name'].dropna().unique()],
                    value=None,
                    multi=True,
                    style={
                        "width": "75%",  # set the width of the dropdown to 50% of its container
                        "height": "50%",  # set the height of each dropdown item to 40 pixels
                        "backgroundColor": colors['background'],  # set the background color of the dropdown to dark gray
                        "color": "#000",  # set the font color of the dropdown to light gray
                        "fontFamily": "sans-serif"  # set the font family of the dropdown to sans-serif
                    }
                )]),
         cyto.Cytoscape(
            id='cytoscape',
            layout={'name': 'circle'},
            style={'height': '700px', 'width': '100%','label': 'data(label)'},
            stylesheet = def_stylesheet
        )
     ])
])

In [6]:
if __name__ == '__main__':
    app.run_server(debug=True)